In [8]:
import openmm, sys
sys.path.append("../../")
from enhanced_sampling import (utils, 
                               system_building as sb, 
                               system_saving as ss,
                               schema)

In [17]:
state_dict = sb.load_input_dir("/Volumes/Rohirrim/local_test/enhanced_sampling/pulling/01.open_to_closed/frame00/",
                               load_psf=False,)

In [10]:
psf_dict = input_dict = sb.load_input_dir("../systems/system00", "../forcefield_data/charmm-params/")

Loading psf from ../systems/system00/step5_input.psf
Loading Charmm params from ../forcefield_data/charmm-params/


In [11]:
psf = input_dict["psf"]

In [13]:
params = schema.SimParams("../sim_params/local_test.yaml")
print(params)
pulling_params = schema.PullingParams("../pulling_methods/cas.yaml")
print(pulling_params)
system = psf.createSystem(input_dict["params"],
                          nonbondedMethod=params.nonbonded_method,
                          constraints=params.constraints,
                          removeCMMotion=False,
                          hydrogenMass=params.hydrogen_mass)

integrator = openmm.LangevinMiddleIntegrator(params.temperature,
                                             params.friction,
                                             params.time_step)

barostat = openmm.MonteCarloMembraneBarostat(params.pressure,
                                             params.surface_tension,
                                             params.temperature,
                                             openmm.MonteCarloMembraneBarostat.XYIsotropic,
                                             openmm.MonteCarloMembraneBarostat.ZFree
                                             )

# for some reason the __init__ won't accept it as an argument, but this works
# the default is 25 timesteps, i've set it for 50
barostat.setFrequency(50)

system.addForce(barostat)

vbond_force = sb.build_virtual_bond(psf, params)

system.addForce(vbond_force)

platform = sb.get_platform_from_params(params)

Loading sim_params from ../sim_params/local_test.yaml
hydrogen_mass       : 4.0 Da
temperature         : 303.15 K
friction            : 1 /ps
time_step           : 0.002 ps
pressure            : 1 bar
surface_tension     : 0
n_steps             : 10
report_freq         : 1
chk_freq            : 10
traj_freq           : 1
virtual_bond        : ['resSeq 46 and name CA and chainid 0', 'resSeq 46 and name CA and chainid 1']
platform            : CPU
precision           : mixed
nonbonded_method    : PME
constraints         : HBonds
steps_dict          : {'Total Time': 10, 'Report Freq': 1, 'Checkpoint Freq': 10, 'Trajectory Freq': 1}
time_dict           : {'Total Time': Quantity(value=0.02, unit=picosecond), 'Report Freq': Quantity(value=0.002, unit=picosecond), 'Checkpoint Freq': Quantity(value=0.02, unit=picosecond), 'Trajectory Freq': Quantity(value=0.002, unit=picosecond)}
n_frames            : 10
Total Time          :	0.00 ns             0.02 ps             20.00 fs            
Report 

In [14]:
sim = openmm.app.Simulation(psf.topology,
                                system=system,
                                integrator=integrator,
                                platform=platform)

In [16]:
for force in system.getForces():
        print(force.getName(), force.getForceGroup())

HarmonicBondForce 0
HarmonicAngleForce 1
HarmonicBondForce 3
PeriodicTorsionForce 2
CustomTorsionForce 4
CMAPTorsionForce 5
NonbondedForce 6
CustomNonbondedForce 6
MonteCarloMembraneBarostat 0
CustomBondForce 0


In [21]:
state=state_dict["state"]

In [22]:
state

<openmm.openmm.State; proxy of <Swig Object of type 'OpenMM::State *' at 0x18458eb10> >

In [23]:
sim.context.setState(state)

OpenMMException: Called setParameter() with invalid parameter name: k

In [29]:
state.getForces()[10]

Quantity(value=Vec3(x=-375.81802341062576, y=2423.5961696812883, z=779.2484754591715), unit=kilojoule/(nanometer*mole))

In [28]:
psfstate = input_dict["state"]
psfstate.getForces()[10]

Quantity(value=Vec3(x=1260.809932197677, y=-147.73494797735475, z=-660.6815036782064), unit=kilojoule/(nanometer*mole))

## see if setting just positions and velocities is different

In [31]:
sim.context.reinitialize()

In [35]:
sim.context.setPositions(state.getPositions())

In [36]:
state02 = sim.context.getState(getForces=True)

In [39]:
state02.getForces()[10]

Quantity(value=Vec3(x=-375.8933460752109, y=2423.488875470829, z=779.2014724831236), unit=kilojoule/(nanometer*mole))